# 09 - Advanced Queries

## Use Case: Complex Filtering & Performance Optimization

As your application grows, you will need queries that go beyond simple filters.
The ORM provides three powerful tools for advanced querying:

- **Subqueries** — Use the result of one query inside another (e.g., "find users who placed high-value orders")
- **Query Cache** — Cache frequently-executed queries with TTL-based expiration
- **Parameterized Filters** — All filter values are automatically bound as query variables for security

This notebook covers:

1. **Subqueries** — Inline sub-SELECT in filters and annotations
2. **Query Cache** — TTL-based caching with auto-invalidation
3. **Parameterized Filters** — How the ORM prevents SurrealQL injection

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Python packages installed (`uv sync` in the project root)

In [ ]:
# Setup: add project root to path and configure the connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

## Model Definitions & Sample Data

We use a User and Order model to demonstrate subqueries, along with
enough sample data to make cache behavior observable.

In [ ]:
# Define User and Order models
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict


class User(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="users")

    id: str | None = None
    name: str
    email: str
    role: str = "user"  # user, admin, moderator


class Order(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="orders")

    id: str | None = None
    user_id: str
    product: str
    total: float
    status: str = "pending"  # pending, completed, cancelled


print("Models defined: User, Order")

In [ ]:
# Create sample users
users_data = [
    User(name="Alice", email="alice@example.com", role="admin"),
    User(name="Bob", email="bob@example.com", role="user"),
    User(name="Charlie", email="charlie@example.com", role="user"),
    User(name="Diana", email="diana@example.com", role="moderator"),
    User(name="Eve", email="eve@example.com", role="user"),
]
created_users = await User.objects().bulk_create(users_data)
print(f"Created {len(created_users)} users")

# Store IDs for reference
alice, bob, charlie, diana, eve = created_users

In [ ]:
# Create sample orders with varying totals
orders_data = [
    # Alice: big spender
    Order(user_id=f"users:{alice.id}", product="Laptop", total=1499.99, status="completed"),
    Order(user_id=f"users:{alice.id}", product="Monitor", total=599.99, status="completed"),
    # Bob: moderate spender
    Order(user_id=f"users:{bob.id}", product="Keyboard", total=79.99, status="completed"),
    Order(user_id=f"users:{bob.id}", product="Mouse", total=29.99, status="completed"),
    # Charlie: big spender
    Order(user_id=f"users:{charlie.id}", product="Gaming PC", total=2499.99, status="completed"),
    # Diana: small orders
    Order(user_id=f"users:{diana.id}", product="USB Cable", total=9.99, status="completed"),
    Order(user_id=f"users:{diana.id}", product="Phone Case", total=19.99, status="pending"),
    # Eve: no completed orders
    Order(user_id=f"users:{eve.id}", product="Headphones", total=149.99, status="cancelled"),
]
created_orders = await Order.objects().bulk_create(orders_data)
print(f"Created {len(created_orders)} orders")

## 1. Subqueries

A `Subquery` wraps a `QuerySet` and embeds it as a sub-SELECT inside another query.
This lets you write queries like "find users whose ID appears in the result of another query."

In SQL terms:
```sql
SELECT * FROM users WHERE id IN (SELECT VALUE user_id FROM orders WHERE total >= 100)
```

The subquery's parameterized variables are automatically remapped to avoid
collisions with the outer query's variables.

In [ ]:
# Find users who placed at least one order over $100 ("VIP users")
from src.surreal_orm import Subquery

# Step 1: Define the inner query — get user_ids from high-value orders
high_value_order_user_ids = Order.objects().filter(
    total__gte=100, status="completed"
).select("user_id")

# Step 2: Use it as a subquery in the outer filter
vip_users = await User.objects().filter(
    id__in=Subquery(high_value_order_user_ids)
).exec()

print("VIP users (orders >= $100):")
for u in vip_users:
    print(f"  {u.name} ({u.email})")

# Expected: Alice ($1499.99, $599.99), Charlie ($2499.99), Eve ($149.99 but cancelled)

In [ ]:
# Subquery with annotate() — scalar sub-SELECT
# annotate() with Subquery uses the GROUP BY path:
#   - With .values("field"), it groups per value
#   - Without .values(), it collapses into a single row (GROUP ALL)

# Example: total completed order revenue as a single scalar
from src.surreal_orm import Count

total_stats = await Order.objects().filter(status="completed").annotate(
    total_orders=Count(),
).exec()

print("Global completed-order stats (GROUP ALL):")
for row in total_stats:
    print(f"  Total completed orders: {row.get('total_orders', 0)}")

# Per-user breakdown: group completed orders by user_id
per_user = await Order.objects().filter(status="completed").values("user_id").annotate(
    order_count=Count(),
).exec()

print("\nCompleted orders per user (GROUP BY user_id):")
for row in per_user:
    print(f"  {row['user_id']}: {row['order_count']} orders")

In [ ]:
# Combine subqueries with other filters
# Find regular users (not admin/moderator) who spent big
big_spender_ids = Order.objects().filter(total__gte=500).select("user_id")

big_spending_regular_users = await User.objects().filter(
    id__in=Subquery(big_spender_ids),
    role="user",  # Only regular users, not admins
).exec()

print("Big-spending regular users:")
for u in big_spending_regular_users:
    print(f"  {u.name} (role: {u.role})")

## 2. Query Cache

The `QueryCache` provides TTL-based caching for query results. When you call
`.cache(ttl=N)` on a QuerySet, the result is cached for N seconds. Subsequent
executions of the same query return the cached result without hitting the database.

The cache is automatically invalidated when:
- A record of the cached table is saved, updated, or deleted (via signals)
- You call `QueryCache.invalidate(ModelClass)` manually
- The TTL expires
- The cache reaches `max_size` (FIFO eviction)

Use cases:
- Dashboard queries that run every few seconds but data changes rarely
- Configuration lookups that almost never change
- Expensive aggregation queries

In [ ]:
# Configure the global query cache
from src.surreal_orm import QueryCache

QueryCache.configure(
    default_ttl=60,    # Default cache duration: 60 seconds
    max_size=500,      # Maximum number of cached queries
    enabled=True,      # Enable caching globally
)

print("QueryCache configured: TTL=60s, max_size=500")

In [ ]:
# Execute a query with caching enabled
import time

# First execution: hits the database
start = time.time()
admins = await User.objects().filter(role="admin").cache(ttl=30).exec()
first_duration = time.time() - start
print(f"First query (DB hit): {len(admins)} admins, took {first_duration:.4f}s")

# Second execution: returns cached result (no DB hit)
start = time.time()
admins_cached = await User.objects().filter(role="admin").cache(ttl=30).exec()
second_duration = time.time() - start
print(f"Second query (cached): {len(admins_cached)} admins, took {second_duration:.4f}s")

if second_duration < first_duration:
    print(f"Cache speedup: {first_duration / max(second_duration, 0.0001):.1f}x faster")

In [ ]:
# Manual cache invalidation
# Invalidate all cached queries for the User table
QueryCache.invalidate(User)
print("Cache invalidated for User table.")

# Next query will hit the database again
admins_fresh = await User.objects().filter(role="admin").cache(ttl=30).exec()
print(f"Fresh query after invalidation: {len(admins_fresh)} admins")

In [ ]:
# Auto-invalidation via signals: saving a User clears the User cache
# Cache a query first
all_users = await User.objects().cache(ttl=30).exec()
print(f"Cached: {len(all_users)} users")

# Create a new user — this triggers post_save, which auto-invalidates the cache
new_user = User(name="Frank", email="frank@example.com", role="user")
await new_user.save()
print("New user created (cache auto-invalidated by post_save signal)")

# Next cached query will hit the DB and return updated results
all_users_updated = await User.objects().cache(ttl=30).exec()
print(f"After invalidation: {len(all_users_updated)} users")

In [ ]:
# Clear the entire cache (all tables)
QueryCache.clear()
print("Entire query cache cleared.")

## 3. Parameterized Filters — Security

Since v0.6.0, **all filter values are automatically parameterized** as query variables.
This means filter values are never interpolated into the query string, preventing
SurrealQL injection attacks.

Instead of:
```sql
SELECT * FROM users WHERE age > 18
```

The ORM generates:
```sql
SELECT * FROM users WHERE age > $_f0
```
with variables: `{"_f0": 18}`

This happens transparently. You do not need to change how you write filters.

In [ ]:
# Demonstrate parameterized filters
# Build a query and inspect the generated SQL + variables
qs = User.objects().filter(role="admin", name__contains="Ali")

# Access the internal compiled query to see parameterization in action
query = qs._compile_query()
print("Generated query:")
print(f"  {query}")
print("\nNotice: filter values are bound as $_fN variables, not embedded in the query.")
print("This prevents SurrealQL injection attacks.")

In [ ]:
# You can also use explicit variable references with .variables()
# This is useful when you want to name your variables
users = await User.objects().filter(
    role="$target_role"  # Reference a named variable
).variables(
    target_role="admin"  # Bind the value
).exec()

print(f"Admins found via explicit variable: {len(users)}")
for u in users:
    print(f"  {u.name} ({u.role})")

In [ ]:
# Even potentially dangerous input is safely parameterized
malicious_input = "admin' OR 1=1 --"

# This is SAFE: the value is bound as a variable, not interpolated
results = await User.objects().filter(role=malicious_input).exec()
print(f"Results with malicious input: {len(results)} (expected 0 — injection prevented)")

## Cleanup

Remove all data created during this notebook.

In [ ]:
# Clean up all tables and reset the cache
QueryCache.clear()
await User.raw_query("DELETE FROM users")
await Order.raw_query("DELETE FROM orders")

print("Cleanup complete: all users, orders deleted, cache cleared.")

## Summary

| Feature | Syntax | When to Use |
|---------|--------|-------------|
| **Subquery (filter)** | `filter(id__in=Subquery(qs))` | Nest one query's results inside another's WHERE clause |
| **Subquery (annotate)** | `annotate(n=Subquery(qs))` | Attach a scalar sub-SELECT to each result |
| **Query Cache** | `.cache(ttl=30).exec()` | Cache frequently-run queries to reduce DB load |
| **Cache invalidation** | `QueryCache.invalidate(Model)` | Manually clear cache for a specific table |
| **Auto-invalidation** | (automatic via signals) | Cache is cleared when records are saved/updated/deleted |
| **Parameterized filters** | (automatic) | All filter values are bound as `$_fN` variables |
| **Explicit variables** | `.filter(f="$var").variables(var=v)` | Named variable bindings for readability |

**Security note**: Parameterized filters are always active. You do not need to opt in.
All user input is safely bound as query variables, never interpolated into the query string.